In [1]:
from cogdl.datasets.grb_data import Cora_GRBDataset
from cogdl.utils import set_random_seed
from cogdl.utils.grb_utils import evaluate, GCNAdjNorm
import copy
import torch
dataset = Cora_GRBDataset()
graph = copy.deepcopy(dataset.get(0))
# device = "cpu"
device = "cuda:0"
device_ids = [0]
graph.to(device)
test_mask = graph.test_mask
set_random_seed(40)

/home/xu-ll18/miniconda3/envs/ml/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# train surrogate model
from cogdl.models.nn import GCN
from cogdl.trainer import Trainer
from cogdl.wrappers import fetch_model_wrapper, fetch_data_wrapper
model_sur = GCN(
    in_feats=graph.num_features,
    hidden_size=64,
    out_feats=graph.num_classes,
    num_layers=2,
    dropout=0.5,
    activation="relu"
)
print(model_sur)
mw_class = fetch_model_wrapper("node_classification_mw")
dw_class = fetch_data_wrapper("node_classification_dw")
optimizer_cfg = dict(
                    lr=0.01,
                    weight_decay=0
                )
model_wrapper = mw_class(model_sur, optimizer_cfg)
dataset_wrapper = dw_class(dataset)
trainer = Trainer(epochs=200,
                  early_stopping=True,
                  patience=50,
                  cpu=device=="cpu",
                  device_ids=device_ids)
trainer.run(model_wrapper, dataset_wrapper)
model_sur.load_state_dict(torch.load("./checkpoints/model.pt"), False)
model_sur.to(device)
test_score = evaluate(model_sur,
                      graph,
                      mask=test_mask,
                      device=device)
print("Test score before attack for surrogate model: {:.4f}.".format(test_score))

GCN(
  (layers): ModuleList(
    (0): GCNLayer(
      (linear): Linear(in_features=302, out_features=64, bias=True)
      (dropout): Dropout(p=0.5, inplace=False)
      (act): ReLU(inplace=True)
    )
    (1): GCNLayer(
      (linear): Linear(in_features=64, out_features=7, bias=True)
    )
  )
)
Model Parameters: 19847


Epoch: 102, train_loss:  0.6244, val_acc:  0.7948:  51%|█████     | 102/200 [00:03<00:03, 27.32it/s]


Saving 53-th model to ./checkpoints/model.pt ...
Loading model from ./checkpoints/model.pt ...
{'test_acc': 0.7972636815920398, 'val_acc': 0.8097014925373134}
Test score before attack for surrogate model: 0.7973.


In [3]:
# train target model
model_target = GCN(
    in_feats=graph.num_features,
    hidden_size=64,
    out_feats=graph.num_classes,
    num_layers=3,
    dropout=0.5,
    activation=None,
    norm="layernorm"
)

In [4]:
from cogdl.attack.injection import FGSM
attack = FGSM(epsilon=0.01,
              n_epoch=10,
              n_inject_max=10,
              n_edge_max=20,
              feat_lim_min=-1,
              feat_lim_max=1,
              device=device,
              verbose=False)

In [5]:
mw_class = fetch_model_wrapper("node_classification_mw")
dw_class = fetch_data_wrapper("node_classification_dw")
optimizer_cfg = dict(
                    lr=0.01,
                    weight_decay=0
                )
model_wrapper = mw_class(model_target, optimizer_cfg)
dataset_wrapper = dw_class(dataset)
trainer = Trainer(epochs=200,
                  early_stopping=True,
                  patience=50,
                  cpu=device=="cpu",
                  attack=attack,
                  attack_mode="injection",
                  device_ids=device_ids)
trainer.run(model_wrapper, dataset_wrapper)
model_target.load_state_dict(torch.load("./checkpoints/model.pt"), False)
model_target.to(device)
test_score = evaluate(model_target,
                      graph,
                      mask=test_mask,
                      device=device)
print("Test score before attack for target model: {:.4f}.".format(test_score))

Model Parameters: 24263


  0%|          | 0/200 [00:00<?, ?it/s]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument tensors in method wrapper___cat)

: 

In [9]:
# FGSM attack
from cogdl.attack.injection import FGSM
# attack = FGSM(epsilon=0.01,
#               n_epoch=1000,
#               n_inject_max=100,
#               n_edge_max=200,
#               feat_lim_min=-1,
#               feat_lim_max=1,
#               device=device)
attack = FGSM(epsilon=0.01,
              n_epoch=10,
              n_inject_max=10,
              n_edge_max=20,
              feat_lim_min=-1,
              feat_lim_max=1,
              device=device,
              verbose=False)
graph_attack = attack.attack(model=model_sur,
                             graph=graph,
                             adj_norm_func=GCNAdjNorm)
graph_attack.y = graph.y
print(graph_attack)

# apply injection attack
test_score_sur = evaluate(model_sur,
                          graph_attack,
                          mask=test_mask,
                          device=device)
print("Test score after attack for surrogate model: {:.4f}.".format(test_score_sur))

# transfer to target model
test_score_target_attack = evaluate(model_target,
                                    graph_attack,
                                    mask=test_mask,
                                    device=device)
print("Test score after attack for target model: {:.4f}.".format(test_score_target_attack))

Epoch 999, Loss: 2.1965, Surrogate test score: 0.0697: 100%|██████████| 1000/1000 [00:39<00:00, 25.43it/s]


Surrogate test score: 0.0697
Attack runtime: 39.5276.
Graph(x=[2780, 302], y=[2680], edge_index=[2, 50296], edge_attr=[50296])
Test score after attack for surrogate model: 0.2488.
Test score after attack for target model: 0.3085.
